### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import time 

import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

def stemmer(text):
    stem = PorterStemmer()
    return ' '.join([stem.stem(w) for w in word_tokenize(text)])

def lemmatizer(text):
    lemm = WordNetLemmatizer()
    return ' '.join([lemm.lemmatize(w) for w in word_tokenize(text)])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sspillane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sspillane\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sspillane\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
df = pd.read_csv('data/dataset.csv')

In [23]:
df.head()

In [29]:
X = df.drop('humor', axis = 1)
y = df['humor']

# Stemmed data
X_stemmed = X['text'].apply(stemmer)
X_train_stemmed, X_test_stemmed, y_train_stemmed, y_test_stemmed = train_test_split(X_stemmed, y, random_state = 42)

# Lemmatized data
X_lemmed = X['text'].apply(lemmatizer)
X_train_lemmed, X_test_lemmed, y_train_lemmed, y_test_lemmed = train_test_split(X_lemmed, y, random_state = 42)


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [30]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'], 
             'best_params': ['', '', ''],
             'best_score': ['', '', '']}).set_index('model')

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


## Count Vectorizer Pipelines

In [ ]:
# Log Reg
log_vect_pipe = Pipeline([
                          ('cvect', CountVectorizer()),
                          ('lgr', LogisticRegression())
                          ])
log_vect_pipe_stemmed.fit(X_train_stemmed, y_train_stemmed)
log_vect_test_acc_stemmed = log_vect_pipe_stemmed.score(X_test_stemmed, y_test_stemmed)

log_vect_pipe_lemmed.fit(X_train_lemmed, y_train_lemmed)
log_vect_test_acc = log_vect_pipe.score(X_test, y_test)


# Decision Tree Classifier
dt_vect_pipe = Pipeline([
                          ('cvect', CountVectorizer()),
                          ('lgr', DecisionTreeClassifier())
                          ])
dt_vect_pipe_stemmed.fit(X_train_stemmed, y_train_stemmed)
dt_vect_test_acc = dt_vect_pipe.score(X_test, y_test)


# Naive Bayes Classifier
bayes_vect_pipe = Pipeline([
                          ('cvect', CountVectorizer()),
                          ('bayes', MultinomialNB())
                          ])
bayes_vect_pipe_stemmed.fit(X_train_stemmed, y_train_stemmed)
bayes_vect_test_acc = bayes_vect_pipe.score(X_test, y_test)

('bayes', MultinomialNB())


In [ ]:
cvect_params = {
                'cvect__max_features': [100, 500, 1000, 2000],
                'cvect__stop_words': ['english', None]
                }

# Count Vectorizer
vect_grid = GridSearchCV(log_vect_pipe, param_grid=cvect_params)
vect_grid.fit(X_train, y_train)
test_acc = vect_grid.score(X_test, y_test)
print(vect_grid.best_params_)

## TF IDF Pipelines

In [ ]:
# Log Reg
log_tfidf_pipe = Pipeline([
                          ('tfidf', TfidfVectorizer()),
                          ('lgr', LogisticRegression())
                          ])
log_tfidf_pipe.fit(X_train, y_train)
log_tfidf_test_acc = log_tfidf_pipe.score(X_test, y_test)


# Decision Tree Classifier
dt_tfidf_pipe = Pipeline([
                          ('tfidf', TfidfVectorizer()),
                          ('lgr', DecisionTreeClassifier())
                          ])
dt_tfidf_pipe.fit(X_train, y_train)
dt_tfidf_test_acc = dt_tfidf_pipe.score(X_test, y_test)


# Naive Bayes Classifier
bayes_tfidf_pipe = Pipeline([
                          ('tfidf', TfidfVectorizer()),
                          ('bayes', MultinomialNB())
                          ])
bayes_tfidf_pipe.fit(X_train, y_train)
bayes_tfidf_test_acc = bayes_tfidf_pipe.score(X_test, y_test)


## Grid Searching

In [ ]:
cvect_params = {
                'cvect__max_features': [100, 500, 1000, 2000],
                'cvect__stop_words': ['english', None]
                }

# Count Vectorizer
vect_grid = GridSearchCV(vect_pipe_1, param_grid=params)
vect_grid.fit(X_train, y_train)
test_acc = vect_grid.score(X_test, y_test)
print(vect_grid.best_params_)


tfidf_params = {
                'tfidf__max_features': [100, 500, 1000, 2000],
                'tfidf__stop_words': ['english', None]
                }


# Count Vectorizer
tfidf_grid = GridSearchCV(vect_pipe_1, param_grid=params)
tfidf_grid.fit(X_train, y_train)
test_acc = tfidf_grid.score(X_test, y_test)
print(tfidf_grid.best_params_)